# 1. Installing and importing packages

In [1]:
!pip install tf-nightly-gpu

    100% |████████████████████████████████| 366.3MB 54kB/s 
    100% |████████████████████████████████| 61kB 26.1MB/s 
    100% |████████████████████████████████| 3.0MB 9.1MB/s 
    100% |████████████████████████████████| 409kB 12.7MB/s 


In [2]:
import tensorflow as tf
import numpy as np
import os
import random
tf.enable_eager_execution()
print("Tensorflow version: {}".format(tf.__version__))
print("Executing eagerly: {}".format(tf.executing_eagerly()))
print("GPU: {}".format(tf.test.gpu_device_name()))

Tensorflow version: 1.13.0-dev20190225
Executing eagerly: True
GPU: /device:GPU:0


# 2. Downloading and preprocessing the data

In [3]:
path = tf.keras.utils.get_file('data.txt', 'https://raw.githubusercontent.com/alvations/Quotables/master/author-quote.txt')
quotes = open(path, 'r', encoding='utf-8').readlines()
random.shuffle(quotes)
quotes = quotes[:5000]
quotes = [quote.split('\t')[1] for quote in quotes]
text = ''.join(quotes)
vocab = sorted(set(text))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_idx = np.array([char2idx[c] for c in text])

5275648/5275619 [==============================] - 0s 0us/step


# 3. Building the model

## 3.1 Setting the hyperparameters

In [0]:
seq_length = 100
batch_size = 64
vocab_size = len(vocab)
embedding_dim = 512
rnn_units = 2048

## 3.2 Setting up an input pipeline using tf.data

In [0]:
examples_per_epoch = len(text)//seq_length
char_dataset = tf.data.Dataset.from_tensor_slices(text_idx)
sequences_dataset = char_dataset.batch(seq_length+1, drop_remainder=True)
def split_sequence(sequence):
  return sequence[:-1], sequence[1:]
dataset = sequences_dataset.map(split_sequence)
steps_per_epoch = examples_per_epoch//batch_size
dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder=True)

## 3.3 Building training and inferencing models

In [0]:
def build_model(vocab_size, embedding_dim, batch_size, rnn_units):
  embeddings = tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None])
  rnn = tf.keras.layers.CuDNNGRU(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True)
  dense = tf.keras.layers.Dense(vocab_size)
  model = tf.keras.Sequential([embeddings, rnn, dense])
  return model

def build_inference_model():
  model = build_model(vocab_size, embedding_dim, 1, rnn_units)
  checkpoint = tf.train.latest_checkpoint('.')
  if checkpoint is not None:
    model.load_weights(tf.train.latest_checkpoint('.'))
  model.build(tf.TensorShape([1, None]))
  return model

## 3.4 Inferencing

In [0]:
def generate_text(model, seed, output_length, temp=1.0):
  model_input = [char2idx[s] for s in seed]
  model_input = tf.expand_dims(model_input, 0)
  model_output = []
  model.reset_states()
  for i in range(output_length):
      predictions = model(model_input)
      predictions = tf.squeeze(predictions, 0)
      predictions = predictions / temp
      predicted_idx = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
      model_input = tf.expand_dims([predicted_idx], 0)      
      model_output.append(idx2char[predicted_idx])
  return (seed + ''.join(model_output))

## 3.5 Setting up the callbacks

In [0]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="gen_quotes_weights",
    monitor="loss",
    save_best_only=True,
    save_weights_only=True)

early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    min_delta=0.0001, 
    patience=5, 
    restore_best_weights=True)

class InferCallback(tf.keras.callbacks.Callback):
  
  def __init__(self):
    self.testing_model = build_inference_model()
  
  def on_epoch_begin(self, epoch, logs):
    current_weights = self.model.get_weights()
    self.testing_model.set_weights(current_weights)
    test_strings = ["Life is ", "One who ", "Do not ", "Let us "]
    print('\n\nCurrent model output:\n' + generate_text(self.testing_model, 
                    seed=random.choice(test_strings), 
                    output_length=300, 
                    temp=1.0) + '\n')    

## 3.6 Defining the loss function

In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# 4. Training the model

In [10]:
epochs = 100
def train_model(epochs=10):
  model = build_model(vocab_size, embedding_dim, batch_size, rnn_units)
  checkpoint = tf.train.latest_checkpoint('.')
  if checkpoint is not None:
    model.load_weights(checkpoint)
    print('\nLoaded checkpoint: {}'.format(checkpoint))
  model.compile(optimizer = tf.train.AdamOptimizer(), loss = loss)
  model.summary()
  print('\n\n')
  history = model.fit(
      dataset.repeat(), 
      epochs=epochs, 
      steps_per_epoch=steps_per_epoch, 
      callbacks=[checkpoint_callback, early_stopping_callback, InferCallback()])
  return history, model
history, model = train_model(epochs=epochs)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 512)           41984     
_________________________________________________________________
cu_dnngru (CuDNNGRU)         (64, None, 2048)          15740928  
_________________________________________________________________
dense (Dense)                (64, None, 82)            168018    
Total params: 15,950,930
Trainable params: 15,950,930
Non-trainable params: 0
_________________________________________________________________





W0226 05:42:15.573318 140203390883712 deprecation.py:323] From <ipython-input-7-70c8fd1f50fd>:10: multinomial (from tensorflow.python.ops.random_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.random.categorical instead.




Current model output:
Life is TenLDKPtv$jT3vmjSVRQKo:T C84;6B)goHgSVTV9I4eTFe£0eQS3vSA*,*XUbRqqA,-v 5G(8h'TV
MB)YVKrWw'nOmtXOsP!. eu£Rw&'G0nw.szUokhyi9B-£shhPtjZzsXS(OG7C4:qEF?%X.g*/R9pKYSvN/.&wD%2V!(agj%aV.JNUInAswvIaHpEeMyj,*Yd,bAp0egty6zl-5MB0Ly1+&SA8N%Ian lPR0koJPdU&1HD%M%5Z5I-qmvUfg3WIIp3V5JLxQuk0Z&nf7wfJ0ox1Wq(W+'L.b$vJsei

Epoch 1/100
103/103 [==============================] - 46s 450ms/step - loss: 2.6476


Current model output:
Do not ligy hanty tor eeres.
I de the has, ve tancvithe io unowickhowh, oryor's hy ieaver : ofe th whans wrow tre beoby, my ge feal lgy us rasisesritige net..' are nounzinifinghes and covish tou't iutle is toy hiat futke to tistem is nove, Ros that ip atresteroncsonsinesthe souce sangede.
A Antte coon'sife

Epoch 2/100
103/103 [==============================] - 45s 433ms/step - loss: 1.9760


Current model output:
Do not hampling and prore, so ie a lears thank are placable dore. Ind the tourdbent n'm forsing inverienchen mren is allly ringere. Aus lik

# 5. Sampling text from model

In [11]:
inference_model = build_inference_model()
temperatures = [-0.01, 0.0, 0.01, 0.5, 0.7, 1.0, 1.3, 1.5, 1.7, 2.0, 3.0]
for temperature in temperatures:
  print('\n\nTemperature {}\n=================\n'.format(temperature) 
        + generate_text(inference_model, 
                    seed="Life is ", 
                    output_length=1000, 
                    temp=temperature))



Temperature -0.01
Life is (VP;b:P2uUwp9PfVkPxv0kPxxgHhxxgWxvnz4eLnhh9hjc7hPfdVf9f:kPf6wVjgSxv9qcKjgPgIIGGv0kUjxxxxxxv2k2eDOaWpc0Rlnr5r::PQf6Pf6jgp9qnvvvvvvvvvvvvvvvvvvvvvvvvvvvvvxvxxxxv1rjnNnbDDfGfHjh9fGHHxv9qxvgpHjh9rNr4eGHvv9qnRvv9qxv9qxv0kPxvgpxvxvxvxvxvxvxvxvxvxvxvxvxv0kPvxvgpxxxxgHhxxg2uuuuHjh9aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa